# parsing_tir

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm.notebook import trange, tqdm
from tqdm.gui import tqdm as tqdm_gui
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display


импорт библиотек

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

опции вывода данных

In [3]:
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

создаем функцию предварительного просмотра данных

In [4]:
oll_url=['https://tiraet.com/catalog/Komplektujuschie/?PAGEN_1=',
         'https://tiraet.com/catalog/2873/?PAGEN_1=',
         'https://tiraet.com/catalog/Kompjuternaja-mebel/?PAGEN_1=',
         'https://tiraet.com/catalog/bytovaya/?PAGEN_1=',
         'https://tiraet.com/catalog/ohrannye-sistemy/?PAGEN_1=',
         'https://tiraet.com/catalog/svet-i-jelektrika/?PAGEN_1=',
         'https://tiraet.com/catalog/orgtehnika/?PAGEN_1=',
         'https://tiraet.com/catalog/PHoto--i-video/?PAGEN_1=',
         'https://tiraet.com/catalog/setevoe-oborudovanie/?PAGEN_1=',
         'https://tiraet.com/catalog/jelektropitanie/?PAGEN_1=',
         'https://tiraet.com/catalog/sport/?PAGEN_1=',
         'https://tiraet.com/catalog/avtojelektronika/?PAGEN_1=',
         'https://tiraet.com/catalog/rashodnye-materialy/?PAGEN_1=',         
]

выбираем начальные страницы для парсинга

In [5]:
j=1
def parsing_2(url):
    data = pd.DataFrame(columns=['title', 'url', 'date', 'availability', 'id', 'price', 'old_price'])    
    for u in url:
        reqx = requests.get(u)
        soup = bs(reqx.text, 'html.parser')
        max_page = soup.findAll('div', class_='nums')
        maxx = int(max_page[0].text.strip().split('\n')[-1])
        for j in tqdm(range(1,maxx + 1)):        
            reqx = requests.get(u + str(j))
            soup = bs(reqx.text, 'html.parser')
            element = soup.findAll('div', class_='item_info')
            datax = []
            for i in element:
                datax.append({'title' : i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm').text.strip(),
                             'url' : 'https://tiraet.com' + i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm')['href'],
                             'date' : pd.to_datetime('today').normalize(),
                             'availability' : i.find('div', class_='item-stock js-show-stores js-show-info-block').text.strip(),
                             'id' : int(i.find('div', class_='article_block').text.replace('Арт.: ','')),
                             'price' : float(i.find('div', class_='price_matrix_wrapper').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper'))) != "<class 'NoneType'>" else np.NaN ,
                             'old_price' : float(i.find('div', class_='price_matrix_wrapper strike_block').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper strike_block'))) != "<class 'NoneType'>" else np.NaN 
                            })
            datax = pd.DataFrame(datax)
            data = pd.concat([data, datax] , ignore_index=True)
    return data

Создаем функцию предварительного парсинга

179  -  16  -  23  -  100  -  33  -  60  -  10  -  4  -  31  -  25  -  5  -  5  -  81 

In [6]:
dft = parsing_2(oll_url)
dft

  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

...

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]

,title,url,date,availability,id,price,old_price
0,Конверт бумажный для CD/DVD на 1 диск Verbatim (49992) 1 шт.,https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/sumki-dlja-diskov/konvert_bumazhnyy_dlya_cd_dvd_na_1_disk_verbatim_49992_1_sht/,2024-08-29,Много,88716,1.0,NaN
1,Переходник для мыши COM в PS/2 (A-235),https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/kabelja-udliniteli-i-perehodniki/Perehodnik-dlja-myshi-COM-v-PS2-A-235/,2024-08-29,Много,284,5.0,NaN
2,"Кабель MINI USB2.0 (A - MINI USB 4PM)1,8м (CC-USB2-AM4P-6)",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/kabelja-udliniteli-i-perehodniki/kabel-mini-usb20-a---mini-usb-4pm18m-cc-usb2-am4p-6/,2024-08-29,Много,8683,5.0,NaN
3,"BOX для CD/DVD Verbatim Slim, на 1 диск 120мм, PVC (49979) Black 1шт",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/sumki-dlja-diskov/box_dlya_cd_dvd_verbatim_slim_na_1_disk_120mm_pvc_49979_black_1sht/,2024-08-29,Много,85078,6.6,NaN
4,"Диск CD-R 700MB VERBATIM, 52X (1шт) EXTRA PROTECTION(43411/43351/43437)",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/diski/disk_cdr_700mb_verbatim_52x_1sht_extra_protection_43411_43351_43437/,2024-08-29,Много,12819,6.6,NaN
...,...,...,...,...,...,...,...
11923,Головка для плоттеров Canon PF-03 (2251B001),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/golovka-dlja-plotterov-canon-pf-03-2251b001/,2024-08-29,Мало,27477,7920.0,NaN
11924,Узел термозакрепления (для сетевых апп)CANON iR2202/2204(FM1-F162-0XX),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/uzel_termozakrepleniya_dlya_setevykh_app_canon_ir2202_2204_fm1_f162_0xx/,2024-08-29,Мало,55309,8085.0,NaN
11925,Головка для плоттеров Canon PF-04 (3630B001),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/golovka_dlya_plotterov_canon_pf_04_3630b001/,2024-08-29,Мало,56144,8250.0,NaN
11926,Картридж EPSON MX-80/LX-300/400/800/850/880 Patron,https://tiraet.com/catalog/rashodnye-materialy/Rashodnye-k-matrichnym-printeram-kartridzhi-lenty/kartridji_matr/kartridzh-epson-lx-300400800850880mx-80/,2024-08-29,Нет в наличии,249,38.0,NaN


dft.to_csv('csv/tiraet_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)  

dft = pd.read_csv('csv/tiraet_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

тестовое сохранение информации после предварительного парсинга

In [7]:
tiraet_group = pd.read_csv('csv/atiraet_group.csv')

Загружаем данные из файла  с информацией о группах и категориях товара по его `id`

In [8]:
tiraet_group = tiraet_group.drop_duplicates(keep= False )

Удаляем дублекаты в загруженном фрейме

In [9]:
dft = dft.merge(tiraet_group, how='left', on='id')

Объединяем датафрейм предварительного парсинга с датафреймом о группах и категориях товара

In [10]:
t_df_new = dft[(dft['_category'].isna()) | (dft['_subcategory'].isna()) | (dft['_group'].isna())]

Отфильтровываем в отдельный датафрейм данные не содержащие информацию о группах и категориях товара

In [11]:
dft = dft[~((dft['_category'].isna()) | (dft['_subcategory'].isna()) | (dft['_group'].isna()))]

Отчищаем основной датафрейм от данных о товарах не содержащих информацию о группах и категориях

In [12]:
def find_groups(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    elem_1 = soup.findAll('a', class_='breadcrumbs__link colored_theme_hover_bg-el-svg')
    elem_2 = soup.findAll('div', class_='breadcrumbs__item')
    try:
        if len(elem_1) <=2:
            if len(elem_2) <= 4:
                d_14 = 'T_'+(elem_1[0].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_15 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_16 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
            else:
                d_14 = 'T_'+(elem_1[0].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_15 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_16 = 'T_'+(elem_2[4].text.replace(',','').replace(' ','_').replace('-','_').lower())
        else:
            d_14 = 'T_'+(elem_1[0].text.replace(',','').replace(' ','_').replace('-','_').lower())
            d_15 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
            d_16 = 'T_'+(elem_1[2].text.replace(',','').replace(' ','_').replace('-','_').lower())
    except:
        d_14 = 'err_'
        d_15 = 'err_'
        d_16 = 'err_'
        
    return pd.Series([ (d_14), (d_15), (d_16)])

Создаем функцию для парсинга данных о группах и категориях товара

In [13]:
tqdm.pandas() 
t_df_new[['_category','_subcategory','_group']] = t_df_new['url'].progress_apply(find_groups)

  0%|          | 0/156 [00:00<?, ?it/s]

Парсим данные о группах и категориях товара

In [14]:
t_df_new._category.unique()

array(['T_компьютерная_техника', 'T_мобильные_телефоны', 'T_для_дома',
       'T_бытовая_техника', 'err_', 'T_оргтехника', 'T_фото_аудио',
       'T_спорт_туризм_увлечения', 'T_автотовары'], dtype=object)

проссматриваем информацию о категориях нового товара

In [15]:
t_df_new._subcategory.unique()

array(['T_аксессуары_компьютерные', 'T_комплектующие', 'T_мультимедиа',
       'T_монитор', 'T_компьютер_ноутбук_планшет',
       'T_аксессуары_к_телефонам', 'T_смартфон', 'T_кухонная_утварь',
       'T_инструменты', 'T_мебель', 'T_техника_для_кухни',
       'T_климатическая_техника', 'T_крупная_бытовая_техника',
       'T_мелкая_бытовая_техника', 'T_гигиена_здоровье_уход', 'err_',
       'T_проекционное_оборудование', 'T_видеотехника', 'T_туризм',
       'T_автохолодильник'], dtype=object)

Просматриваем информацию о подкатегориях нового товара

In [16]:
t_df_new._group.unique()

array(['T_бокс_для_диска', 'T_кабель_удлинитель_переходник',
       'T_привод_оптический', 'T_диск_дискета', 'T_наушники',
       'T_система_охлаждения', 'T_монитор', 'T_блоки_питания_пк',
       'T_ноутбук', 'T_чехол', 'T_смартфон', 'T_посуда', 'T_штопор',
       'T_нож_и_точилка', 'T_электроинструмент',
       'T_измерительный_инструмент', 'T_ручной_инструмент', 'T_кресло',
       'T_вспениватель_капучинатор', 'T_бутербродница_блинница',
       'T_электрочайник', 'T_вентилятор', 'T_соковыжималка',
       'T_швейная_машина', 'T_часы', 'T_мультиварка',
       'T_фен_щипцы_и_щетка', 'T_электропечь_плита', 'T_вытяжка',
       'T_пылесос', 'T_микроволновая_печь', 'T_морозильник',
       'T_духовой_шкаф', 'T_кондиционер', 'err_', 'T_холодильник',
       'T_варочная_панель', 'T_плита', 'T_экран_для_проектора',
       'T_видеокамера_gopro', 'T_сумка_холодильник', 'T_автохолодильник'],
      dtype=object)

Просматриваем информацию о группах нового товара

In [17]:
t_data = t_df_new[~(t_df_new['_category']=='err_')].reset_index(drop=True)
t_data = t_data[['id', '_category','_subcategory','_group' ]]
t_data = t_data.sort_values(['_category','_subcategory', '_group'])

Выделяем информацию о группах и категориях нового товара в отдельную переменную и сортируем ее

In [18]:
tiraet_group = pd.concat([tiraet_group, t_data], ignore_index=True).sort_values(['_category','_subcategory', '_group']).reset_index(drop=True)
tiraet_group.to_csv('csv/atiraet_group.csv', index=False)  

Обединяем информацию о группах и категориях товара в один датафрейм и сохраняем ее в старый файл

In [19]:
dft = pd.concat([dft, t_df_new], ignore_index=True).sort_values(['_category','_subcategory', '_group']).reset_index(drop=True)

Объединяем очищенный датафрейм с информацией о товарах и группах товаровс новым датафреймом о товарах и группах товаров

dft.to_csv('csv/tiraet_final_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)  

тестовое сохранение полученной информации

In [20]:
dft[dft['_category']=='err_'].id.count()

1

смотрим количество данных спарсенное с ошибкой

In [21]:
dft[dft['_category']=='err_']

,title,url,date,availability,id,price,old_price,_category,_subcategory,_group
11927,"Музыкальный центр PANASONIC SC-AKX320GSK (2.0, 450Вт, CD, FM, USB, SD, BT, Д/У) BLACK",https://tiraet.com/catalog/bytovaya/melkaja-bytovaja-tehnika/muzykalnye-centry/muzykalnyy_tsentr_panasonic_sc_akx320gsk_2_0_450vt_cd_fm_usb_sd_bt_d_u_black/,2024-08-29,Мало,87154,3799.0,NaN,err_,err_,err_


просматриваем данные спарсенные с ошибкой

In [22]:
dft['error_group'] = 0
dft.loc[dft['_category']=='err', 'error_group'] = 1

В датафрейме создаем столбец данные в котором равны "1", если данные о группе товаров спарсены с ошибкой

In [23]:
for i in dft['_category'].unique():
    t = dft[dft['_category']==i]
    print(i)
    print(t['_subcategory'].unique())
    print('-'*100)

T_автотовары
['T_автомагнитола' 'T_автопылесос' 'T_авторегистратор' 'T_автохолодильник'
 'T_динамики' 'T_компрессоры' 'T_крепления_смартфонов/планшетов'
 'T_модуляторы_fm' 'T_навигаторы_gps']
----------------------------------------------------------------------------------------------------
T_безопасность
['T_видеонаблюдение' 'T_домофоны' 'T_монтаж_и_питание'
 'T_оборудование_poe' 'T_сигнализация_охранно_пожарная' 'T_скуд'
 'T_умный_дом']
----------------------------------------------------------------------------------------------------
T_бытовая_техника
['T_гигиена_здоровье_уход' 'T_климатическая_техника'
 'T_крупная_бытовая_техника' 'T_мелкая_бытовая_техника' 'T_телевизоры'
 'T_техника_для_кухни' 'T_часы_наручные']
----------------------------------------------------------------------------------------------------
T_для_дома
['T_бытовая_химия' 'T_запасные_части' 'T_инструменты' 'T_крепления'
 'T_кухонная_утварь' 'T_мебель' 'T_текстиль' 'T_хозяйственный_инвентарь']
-----------------

# Аналитика акционных товаров

In [24]:
brend = pd.read_excel('csv/brend.xlsx')

In [25]:
def lower(x):
    return x.lower()
brend['names'] = brend['name'].apply(lower)
brend_up = brend.name.unique()
brend_low = brend.names.unique()

Создаем список брендов

In [26]:
def brends(x):
    xx = x.replace('\xa0', ' ')
    xx = xx.split(' ')    
    for i in range(len(xx)-1):
        if xx[i].lower() in brend_low:
            return xx[i].lower()
        elif (str(xx[i].lower())+' '+str(xx[i+1].lower())) in brend_low:
            return (str(xx[i].lower())+' '+str(xx[i+1].lower()))    
    else:    
        return 'другой'
tqdm.pandas()
dft['brend'] = dft['title'].progress_apply(brends)

  0%|          | 0/11928 [00:00<?, ?it/s]

создаем столбец с брендами

In [27]:
dft['action_sale'] = 0
dft.loc[dft['old_price'] > 0,'action_sale'] = 1

Создаем столбец с флагом акции

In [28]:
dft['sale'] = (100 - ((dft['price']*100)/dft['old_price'])).round(2) 

создаем столбец показывающий скидку в процентах

In [29]:
def count_group(x):
    return dft[dft['_group']==x]['id'].count()    
tqdm.pandas()
dft['count_group'] = dft['_group'].progress_apply(count_group)

  0%|          | 0/11928 [00:00<?, ?it/s]

создание столбца с количеством товара в группе

In [30]:
def count_group_presence(x):
    return dft[(dft['availability'].isin(['Мало','Достаточно','Много']))&(dft['_group']==x)]['id'].count()   
tqdm.pandas()
dft['activ_count_group'] = dft['_group'].progress_apply(count_group_presence)

  0%|          | 0/11928 [00:00<?, ?it/s]

создание столбца с количеством товара, который есть в наличии, в группе

In [31]:
def dif_price(x):    
    max_price = dft[(dft['availability'].isin(['Мало','Достаточно','Много']))&(dft['_group']==x)]['price'].max()
    min_price = dft[(dft['availability'].isin(['Мало','Достаточно','Много']))&(dft['_group']==x)]['price'].min() 
    return max_price / min_price
tqdm.pandas()
dft['dif_price'] = dft['_group'].progress_apply(dif_price)

  0%|          | 0/11928 [00:00<?, ?it/s]

создам столбец с отношением самой дорогой цены в группе к самой дешевой.

In [32]:
dft['group_prise'] = dft.groupby('_group')['price'].transform(lambda x: pd.cut(x, bins = 4, labels=["бюджет", "стандарт", 'дорогой', "элитный"]))

создание столбца с ценовой категорией товара

In [33]:
dft = dft[['url', 'id', '_category', '_subcategory', '_group', 'brend', 
         'title', 'date', 'price', 'old_price', 'action_sale', 'sale', 'group_prise', 'dif_price',
         'availability', 'count_group', 'activ_count_group', 'error_group']]

In [34]:
dft['tir'] = 1

In [35]:
dft = dft.drop_duplicates().reset_index(drop=True)

In [36]:
preprocessing(dft)

таблица имеет следующий вид:


,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir
0,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_navitel_rd5_4x50vt_msd_2_usb_bluetooth/,80126,T_автотовары,T_автомагнитола,T_автомагнитола,navitel,"Автомагнитола NAVITEL RD5 (4x50Вт, mSD, 2*USB, Bluetooth)",2024-08-29,660.0,NaN,0,NaN,бюджет,3.331818,Достаточно,9,9,0,1
1,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_sencor_sct_3018mr_fm_am_cd_mp3_wma_usb_aux_4_40w_iso_rca_1din/,78604,T_автотовары,T_автомагнитола,T_автомагнитола,sencor,"Автомагнитола Sencor SCT 3018MR (FM/AM, CD, MP3/WMA, USB, AUX, 4*40W, ISO/RCA, 1DIN)",2024-08-29,660.0,NaN,0,NaN,бюджет,3.331818,Мало,9,9,0,1
2,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_akai_ca016a_9008u_bluetooth_fm_am_mp3_usb_sd_aux_4_25w_1din_black_4905192533076/,83899,T_автотовары,T_автомагнитола,T_автомагнитола,akai,"Автомагнитола AKAI CA016A-9008U (Bluetooth, FM/AM, MP3, USB/SD, AUX, 4*25W,1DIN) black 4905192533076",2024-08-29,759.0,NaN,0,NaN,бюджет,3.331818,Достаточно,9,9,0,1
3,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola_kenwood_kdc_130ug_fm_am_mp3_wav_wma_flac_cd_usb_aux_4_50w_iso_rca_1din_black/,78235,T_автотовары,T_автомагнитола,T_автомагнитола,kenwood,"Автомагнитола Kenwood KDC-130UG (FM/AM, MP3/WAV/WMA/FLAC, CD, USB, AUX, 4*50W, ISO/RCA, 1DIN) black",2024-08-29,1419.0,NaN,0,NaN,стандарт,3.331818,Мало,9,9,0,1
4,https://tiraet.com/catalog/avtojelektronika/avtomagnitoly/avtomagnitola-pioneer-mvh-29bt-4x50vt-mp3wmawavflac-fmam-usb-1-din-bluetooth-isorca/,63822,T_автотовары,T_автомагнитола,T_автомагнитола,pioneer,"Автомагнитола Pioneer MVH-29BT 4x50Вт, MP3/WMA/WAV/FLAC, FM/AM, USB, 1 DIN, Bluetooth, ISO/RCA",2024-08-29,1699.0,NaN,0,NaN,дорогой,3.331818,Мало,9,9,0,1


****************************************************************************************************
таблица имеет 11928 сторок и 19 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                      0
id                       0
_category                0
_subcategory             0
_group                   0
brend                    0
title                    0
date                     0
price                    0
old_price            10115
action_sale              0
sale                 10115
group_prise              0
dif_price                2
availability             0
count_group              0
activ_count_group        0
error_group              0
tir                      0
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url                   0.0
id                    0.0
_category             0.0
_subcategory          0.0
_group                0.0
brend                 0.0
title                 0.0
date                  0.0
price                 0.0
old_price            84.8
action_sale           0.0
sale                 84.8
group_prise           0.0
dif_price             0.0
availability          0.0
count_group           0.0
activ_count_group     0.0
error_group           0.0
tir                   0.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                          object
id                           object
_category                    object
_subcategory                 object
_group                       object
brend                        object
title                        object
date                 datetime64[ns]
price                       float64
old_price                   float64
action_sale                   int64
sale                        float64
group_prise                category
dif_price                   float64
availability                 object
count_group                   int64
activ_count_group             int64
error_group                   int64
tir                           int64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11928 entries, 0 to 11927
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   url                11928 non-null  object        
 1   id                 11928 non-null  object        
 2   _category          11928 non-null  object        
 3   _subcategory       11928 non-null  object        
 4   _group             11928 non-null  object        
 5   brend              11928 non-null  object        
 6   title              11928 non-null  object        
 7   date               11928 non-null  datetime64[ns]
 8   price              11928 non-null  float64   

None

# Парсинг Hi

In [37]:
def h_find_id(x):
    xxx = x.find('span', class_="ty-control-group__item").text  
    return xxx

функция создания столбца с кодом товара

In [38]:
def h_find_availability(t):
    xxx = (t.find('a', class_="ty-control-group product-list-field cm-external-click")
           .text.replace('\ue924','').replace('\ue86c','').replace('\n',''))
    return xxx  

функция создания столбца с парсингом наличия товара

In [39]:
def h_find_title(t):
    xxx = t.find('a', class_="product-title").text
    return xxx

функция создающая столбец с названием товара

In [40]:
def h_find_price(t):
    xxx = t.find('span', class_="ty-price-num").text.replace(u'\xa0',u'')
    return xxx

функция создающая столбец с продажной ценой товара

In [41]:
def h_find_old_price(t):
    try:
        xxx = t.find('span', class_="ty-strike").text.replace(u'\xa0',u'').replace(u'руб',u'')
        return xxx
    except:
        return np.NaN

функция созжающая столбец с предидущей ценой для акционного товара

In [42]:
def h_find_url(t):
    xxx = t.find('a', class_="abt-single-image")['href']
    return xxx

Функция создающая столбец с урл адресом товара

In [43]:
def h_parsing_2(url):
    data = pd.DataFrame(columns=['url', 'id', 'date', 'availability', 'title', 'price', 'old_price'])
    for i in tqdm(url):        
        q = 1
        attempt = 1
        while q>0 or attempt<4:             
            reqx = requests.get(i + str(q))
            soup = bs(reqx.text, 'html.parser')
            datax = pd.DataFrame({'url':pd.Series(soup.findAll('div', class_='ypi-grid-list__item_body'),dtype = 'object')})
            data = pd.concat([data, datax], ignore_index=True)
            if len(list(soup.findAll('div', class_='ypi-grid-list__item_body'))) > 0:
                q = q + 1
                attempt = 1                
            else:
                if attempt == 3:                    
                    q = 0
                    attempt += 1
                else:
                    attempt += 1                    
    data['id'] = data['url'].apply(h_find_id)
    data['date'] = pd.to_datetime('today').normalize()
    data['availability'] = data['url'].apply(h_find_availability)
    data['title'] = data['url'].apply(h_find_title)
    data['price'] = data['url'].apply(h_find_price)
    data['old_price'] = data['url'].apply(h_find_old_price)
    data['url'] = data['url'].apply(h_find_url)
    data['price'] = data['price'].astype(float)
    data.loc[data['old_price'] == '','old_price'] = np.nan
    data['old_price'] = data['old_price'].astype(float)
    return data

Функция предварительного парсинга сайта, которая проходит по всем страницам в урл адресах переменной `all_url` и создает датафрейм в котором храняться: урл адреса товаров, индентефикатор товаров, дата предварительного парсинга товара, наличия товара, названия товара, цены товара, старой цены для акционного товара.

# Парсинг цен кодов товара и url

In [44]:
all_url = ['https://hi-tech.md/televizory-i-elektronika/page-',
           'https://hi-tech.md/bytovaya-tehnika/page-',
           'https://hi-tech.md/kompyuternaya-tehnika/page-',
           'https://hi-tech.md/mebel-tekstil/kompyuternaya-mebel/ofisnye-kresla-i-stulya/komp.-kresla/page-',
           'https://hi-tech.md/instrumenty-i-oborudovanie/elektroinstrument/page-',
           'https://hi-tech.md/tovary-dlya-doma/page-'
          ]

переменная со списком ссылок на сайт для парсинга

In [45]:
dfh = h_parsing_2(all_url)

  0%|          | 0/6 [00:00<?, ?it/s]

предварительный парсинг

In [46]:
dfh = dfh.drop_duplicates().reset_index(drop=True)

удаление дублекатов

dfh.to_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)

создание резервной копии парсинга данных

dfh = pd.read_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

In [47]:
preprocessing(dfh)

таблица имеет следующий вид:


,url,id,date,availability,title,price,old_price
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr1220-01/,Т-000088228,2024-08-29,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN
1,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2016-01/,Т-000088229,2024-08-29,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN
2,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aa4-01/,Т-000088221,2024-08-29,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN
3,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aaa4-01/,Т-000088222,2024-08-29,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN
4,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2025-01/,Т-000088230,2024-08-29,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN


****************************************************************************************************
таблица имеет 9170 сторок и 7 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                0
id                 0
date               0
availability       0
title              0
price              0
old_price       6864
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url              0.0
id               0.0
date             0.0
availability     0.0
title            0.0
price            0.0
old_price       74.9
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                     object
id                      object
date            datetime64[ns]
availability            object
title                   object
price                  float64
old_price              float64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9170 entries, 0 to 9169
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   url           9170 non-null   object        
 1   id            9170 non-null   object        
 2   date          9170 non-null   datetime64[ns]
 3   availability  9170 non-null   object        
 4   title         9170 non-null   object        
 5   price         9170 non-null   float64       
 6   old_price     2306 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 501.6+ KB


None

In [48]:
hai_group = pd.read_csv('csv/ahay_group.csv')

загрузка таблицы с группами товара

In [49]:
hai_group = hai_group.drop_duplicates(keep=False )

In [50]:
len(hai_group)

10504

In [51]:
dfh = dfh.merge(hai_group, how='left', on='id')

обединение таблиц предварительного парсинга и таблицы групп товаров

In [52]:
dfh

,url,id,date,availability,title,price,old_price,_category,_subcategory,_group,_subgroup
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr1220-01/,Т-000088228,2024-08-29,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства
1,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2016-01/,Т-000088229,2024-08-29,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства
2,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aa4-01/,Т-000088221,2024-08-29,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства
3,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aaa4-01/,Т-000088222,2024-08-29,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства
4,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2025-01/,Т-000088230,2024-08-29,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN,H_телевизоры_и_электроника,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства,H_батарейки_и_зарядные_устройства
...,...,...,...,...,...,...,...,...,...,...,...
9165,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-4pl-satipo-transparent-led15w-e27-5628-tkl/,Т-000085036,2024-08-29,В наличии,Люстра 4PL SATIPO TRANSPARENT LED15W E27 (5628) (TKL),2869.0,4099.0,H_товары_для_дома,H_освещение,H_люстры,H_люстры
9166,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-4pl-satipo-amber-led15w-e27-5625-tkl/,Т-000085037,2024-08-29,В наличии,Люстра 4PL SATIPO AMBER LED15W E27 (5625) (TKL),2897.0,4139.0,H_товары_для_дома,H_освещение,H_люстры,H_люстры
9167,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-3pl-universo-black-led15w-e27-4483-tkl/,Т-000085021,2024-08-29,В наличии,Люстра 3PL UNIVERSO BLACK LED15W E27 (4483) (TKL),2939.0,4199.0,H_товары_для_дома,H_освещение,H_люстры,H_люстры
9168,https://hi-tech.md/tovary-dlya-doma/osveschenie/lyustry/lyustra-6pl-fairy-black-6w-g9-4856-tkl/,Т-000085033,2024-08-29,В наличии,Люстра 6PL FAIRY BLACK 6W G9 (4856) (TKL),3037.0,4339.0,H_товары_для_дома,H_освещение,H_люстры,H_люстры


In [53]:
h_df_new = dfh[(dfh['_category'].isna()) | (dfh['_subcategory'].isna()) | (dfh['_group'].isna()) | (dfh['_subgroup'].isna())]

создание тыблицы с товарами, у которых нет данных о группах

In [54]:
len(h_df_new)

96

In [55]:
dfh = dfh[~((dfh['_category'].isna()) | (dfh['_subcategory'].isna()) | (dfh['_group'].isna()) | (dfh['_subgroup'].isna()))]

отчистка таблицы парсинга от пропусков

In [56]:
len(dfh[(dfh['_category'].isna()) | (dfh['_subcategory'].isna()) | (dfh['_group'].isna()) | (dfh['_subgroup'].isna())])

0

dfh = dfh.sample(30)

тестовое уменьшение таблици

In [57]:
def h_xxx(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    elem_2 = soup.findAll('div', class_='ty-breadcrumbs clearfix')
    name = soup.findAll('span', class_='ty-breadcrumbs__current')
            
    try:
        names = name[0].text
        name_group = elem_2[0].text.replace('/'+str(names),'').replace('\n','')
        err = 'err'
    except:
        names = 'Noname'
        name_group = 'Noname/Noname/Noname/Noname/Noname/Noname'
        err = 'Noname_err'
    try:
        len_name = len((elem_2[0].text.replace('/'+str(names),'').replace('\n','')).split('/'))
    except:
        len_name = 1
        
    try:
        if len_name <= 1:
            d_14 = 'err1'
            d_15 = 'err1'
            d_16 = 'err1'
            d_17 = 'err1'
        elif len_name <= 2:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_17 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()      
        elif len_name <= 3:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_17 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
        elif len_name == 4:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower() 
            d_17 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower()
        elif len_name > 4:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower() 
            d_17 = 'H_'+(name_group.split("/")[4]).replace(',','').replace(' ','_').replace('-','_').lower()
    except:
        d_14 = 'err0'
        d_15 = 'err0'
        d_16 = 'err0'
        d_17 = 'err0'
        
    return pd.Series([(d_14), (d_15), (d_16), (d_17)])

Функция основного парсинга, которая для каждой строки по урл адресу парсит и находит наличие товара в магахинах а также группу товара, категорию товара, подкатегорию товата и количество подкатегорий товара

# Парсинг групп товара

In [ ]:
if len(h_df_new) > 0:
    tqdm.pandas() 
    h_df_new[['_category','_subcategory','_group','_subgroup']] = h_df_new['url'].progress_apply(h_xxx)
else:
    pass

  0%|          | 0/96 [00:00<?, ?it/s]

Основной парсинг

In [104]:
h_data = h_df_new[~((h_df_new['_category']=='err1') | (h_df_new['_category']=='err0'))].reset_index(drop=True)
h_data = h_data[['id', '_category','_subcategory','_group', '_subgroup']]
h_data = h_data.sort_values(['_category','_subcategory', '_group', '_subgroup'])

In [105]:
len(h_data)

88

создание таблици с группами нового товара

In [106]:
hai_group = pd.concat([hai_group, h_data], ignore_index=True).sort_values(['_category','_subcategory', '_group', '_subgroup']).reset_index(drop=True)
hai_group.to_csv('csv/ahay_group.csv', index=False)  

добавление в таблицу с группами товара новых данных

In [107]:
dfh = pd.concat([dfh, h_df_new], ignore_index=True).sort_values(['_category','_subcategory', '_group', '_subgroup']).reset_index(drop=True)

объединение таблици со старыми и новыми товарами

In [108]:
dfh._category.unique()

array(['H_no_group', 'H_бытовая_техника', 'H_компьютерная_техника',
       'H_мебель_текстиль', 'H_по_умолчанию', 'H_стройка_и_ремонт',
       'H_телевизоры_и_электроника', 'H_товары_для_дома', 'err1'],
      dtype=object)

In [109]:
dfh._subcategory.unique()

array(['H_no_group', 'H_товар_без_акции', 'H_товар_по_акции',
       'H_аксессуары_для_мелкой_бытовой_техники',
       'H_климатическая_техника', 'H_крупная_техника_для_кухни',
       'H_мелкая_техника_для_кухни', 'H_минимойки',
       'H_приготовление_напитков', 'H_приготовление_пищи',
       'H_пылесосы_и_аксессуары', 'H_стиральные_и_сушильные_машины',
       'H_техника_karcher', 'H_техника_для_здоровья',
       'H_техника_для_красоты', 'H_товары_для_ухода_за_одеждой',
       'H_установка_бытовой_техники', 'H_apple', 'H_аксессуары',
       'H_аудиосистемы', 'H_видеотехника', 'H_гаджеты',
       'H_готовые_решения', 'H_игровые_приставки', 'H_комплектующие',
       'H_крепления', 'H_мониторы', 'H_мультимедиа', 'H_накопители',
       'H_оргтехника', 'H_периферийные_устройства',
       'H_проекторы_и_экраны', 'H_расходные_материалы',
       'H_сетевое_оборудование', 'H_смартфоны_и_аксессуары',
       'H_электропитание', 'H_мебель_для_офиса', 'H_по_умолчанию',
       'H_техника_для_дома',

In [110]:
dfh._group.unique()

array(['H_no_group', 'H_товар_без_акции', 'H_товар_по_акции',
       'H_аксессуары_для_зубных_щеток',
       'H_аксессуары_для_электрических_бритв', 'H_бойлеры',
       'H_вентиляторы', 'H_комплектующие_для_климатической_техники',
       'H_кондиционеры', 'H_кондиционеры_кассетные',
       'H_кондиционеры_мульти_сплит', 'H_кондиционеры_промышленные',
       'H_котлы_газовые', 'H_обогреватели', 'H_отопительные_печи',
       'H_увлажнители_комплексы', 'H_варочные_поверхности', 'H_вытяжки',
       'H_духовые_шкафы', 'H_кронштейны_для_микроволновых_печей',
       'H_микроволновые_печи', 'H_морозильники_и_лари',
       'H_плиты_газовые_и_электрические', 'H_посудомоечные_машины',
       'H_холодильники', 'H_электропечи_и_духовки', 'H_блендеры',
       'H_вакууматоры', 'H_весы_кухонные', 'H_генераторы_льда',
       'H_кухонные_комбайны', 'H_ломтерезки', 'H_миксеры',
       'H_соковыжималки', 'H_соковыжималки_для_цитрусов', 'H_тостеры',
       'H_чопперы', 'H_электромясорубки', 'H_минимойки', 

просмотр категорий, подкатегорий и групп товаров

In [111]:
dfh[dfh['_category']=='err1'].id.count()

8

In [112]:
dfh[dfh['_category']=='err0'].id.count()

0

In [113]:
dfh[dfh['_category']=='err1']

,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir,_subgroup
9258,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energizer-alkaline-lr06-aa-4up/,Т-000094958,err1,err1,err1,NaN,Батарейка Energizer Alkaline LR06 AA 4уп,2024-08-29,12.0,NaN,NaN,NaN,NaN,NaN,В наличии,NaN,NaN,NaN,NaN,err1
9259,https://hi-tech.md/televizory-i-elektronika/televizory/televizor-hisense-65e7nq-pro/,Т-000094928,err1,err1,err1,NaN,телевизор Hisense 65E7NQ PRO,2024-08-29,15599.0,NaN,NaN,NaN,NaN,NaN,В наличии,NaN,NaN,NaN,NaN,err1
9260,https://hi-tech.md/bytovaya-tehnika/tehnika-dlya-krasoty/vesy-napolnye/vesy-polaris-pws-1885dg-180kg/,Т-000063843,err1,err1,err1,NaN,весы Polaris PWS 1885DG 180кг,2024-08-29,340.0,NaN,NaN,NaN,NaN,NaN,Ожидается,NaN,NaN,NaN,NaN,err1
9261,https://hi-tech.md/bytovaya-tehnika/stiralnye-i-sushilnye-mashiny/stiralno-sushilnye-mashiny/stiralnaya-mashina-s-sushkoy-samsung-wd80t554cbt-ua/,Т-000095158,err1,err1,err1,NaN,Стиральная машина с сушкой SAMSUNG WD80T554CBT/UA,2024-08-29,14599.0,NaN,NaN,NaN,NaN,NaN,Ожидается,NaN,NaN,NaN,NaN,err1
9262,https://hi-tech.md/tovary-dlya-doma/servirovka-stola/stolovye-servizy/luminarc-diwali-noir-bulonnica-0-75l-4555-ucenka/,Т-000095044,err1,err1,err1,NaN,"Luminarc DIWALI Noir бульонница 0,75л (4555) уценка",2024-08-29,39.0,43.0,NaN,NaN,NaN,NaN,В наличии,NaN,NaN,NaN,NaN,err1
9263,https://hi-tech.md/tovary-dlya-doma/bytovaya-himiya/sredstva-dlya-stirki/opolaskivatel-softlan-vanilla-and-orchidee1l-0329/,Т-000095102,err1,err1,err1,NaN,Ополаскиватель Softlan Vanilla&Orchidee1л 0329,2024-08-29,46.0,NaN,NaN,NaN,NaN,NaN,Ожидается,NaN,NaN,NaN,NaN,err1
9264,https://hi-tech.md/tovary-dlya-doma/prigotovlenie-pischi/kovshi/kovsh-vinzer-stone-induction-line-50443-1-2l-kryshka/,Т-000095131,err1,err1,err1,NaN,"ковш Vinzer Stone Induction Line 50443 1,2л +крышка",2024-08-29,221.0,NaN,NaN,NaN,NaN,NaN,Ожидается,NaN,NaN,NaN,NaN,err1
9265,https://hi-tech.md/tovary-dlya-doma/kuhonnye-prinadlezhnosti/nozhi/nozh-vinzer-50315-17-8sm-santoku/,Т-000095130,err1,err1,err1,NaN,"нож Vinzer 50315 17,8см сантоку",2024-08-29,238.0,NaN,NaN,NaN,NaN,NaN,Ожидается,NaN,NaN,NaN,NaN,err1


In [114]:
dfh[dfh['_category']=='err0']

,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir,_subgroup


просмотр товаров с ошибками

In [115]:
preprocessing(dfh)

таблица имеет следующий вид:


,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir,_subgroup
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energizer-alkaline-lr06-aa-4up/,Т-000094958,H_no_group,H_no_group,H_no_group,energizer,Батарейка Energizer Alkaline LR06 AA 4уп,2024-08-29,12.0,NaN,0.0,NaN,бюджет,1299.916667,В наличии,8.0,3.0,1.0,0.0,NaN
1,https://hi-tech.md/televizory-i-elektronika/televizory/televizor-hisense-65e7nq-pro/,Т-000094928,H_no_group,H_no_group,H_no_group,hisense,телевизор Hisense 65E7NQ PRO,2024-08-29,15599.0,NaN,0.0,NaN,элитный,1299.916667,В наличии,8.0,3.0,1.0,0.0,NaN
2,https://hi-tech.md/bytovaya-tehnika/tehnika-dlya-krasoty/vesy-napolnye/vesy-polaris-pws-1885dg-180kg/,Т-000063843,H_no_group,H_no_group,H_no_group,polaris,весы Polaris PWS 1885DG 180кг,2024-08-29,340.0,NaN,0.0,NaN,бюджет,1299.916667,Ожидается,8.0,3.0,1.0,0.0,NaN
3,https://hi-tech.md/bytovaya-tehnika/stiralnye-i-sushilnye-mashiny/stiralno-sushilnye-mashiny/stiralnaya-mashina-s-sushkoy-samsung-wd80t554cbt-ua/,Т-000095158,H_no_group,H_no_group,H_no_group,samsung,Стиральная машина с сушкой SAMSUNG WD80T554CBT/UA,2024-08-29,14599.0,NaN,0.0,NaN,элитный,1299.916667,Ожидается,8.0,3.0,1.0,0.0,NaN
4,https://hi-tech.md/tovary-dlya-doma/servirovka-stola/stolovye-servizy/luminarc-diwali-noir-bulonnica-0-75l-4555-ucenka/,Т-000095044,H_no_group,H_no_group,H_no_group,luminarc,"Luminarc DIWALI Noir бульонница 0,75л (4555) уценка",2024-08-29,39.0,43.0,1.0,9.3,бюджет,1299.916667,В наличии,8.0,3.0,1.0,0.0,NaN


****************************************************************************************************
таблица имеет 9266 сторок и 20 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                     0
id                      0
_category               0
_subcategory            0
_group                  0
brend                  96
title                   0
date                    0
price                   0
old_price            6921
action_sale            96
sale                 6960
group_prise            96
dif_price             146
availability            0
count_group            96
activ_count_group      96
error_group            96
tir                    96
_subgroup            9170
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url                   0.0
id                    0.0
_category             0.0
_subcategory          0.0
_group                0.0
brend                 1.0
title                 0.0
date                  0.0
price                 0.0
old_price            74.7
action_sale           1.0
sale                 75.1
group_prise           1.0
dif_price             1.6
availability          0.0
count_group           1.0
activ_count_group     1.0
error_group           1.0
tir                   1.0
_subgroup            99.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                          object
id                           object
_category                    object
_subcategory                 object
_group                       object
brend                        object
title                        object
date                 datetime64[ns]
price                       float64
old_price                   float64
action_sale                 float64
sale                        float64
group_prise                category
dif_price                   float64
availability                 object
count_group                 float64
activ_count_group           float64
error_group                 float64
tir                         float64
_subgroup                    object
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9266 entries, 0 to 9265
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   url                9266 non-null   object        
 1   id                 9266 non-null   object        
 2   _category          9266 non-null   object        
 3   _subcategory       9266 non-null   object        
 4   _group             9266 non-null   object        
 5   brend              9170 non-null   object        
 6   title              9266 non-null   object        
 7   date               9266 non-null   datetime64[ns]
 8   price              9266 non-null   float64     

None

просмотр данных о таблице

dfh.to_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)

создание резервной копии парсинга данных

dfh = pd.read_csv('csv/hi_tech_test_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

загрузка резервной копии парсинга данных

In [116]:
dfh.head(2)

,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir,_subgroup
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energizer-alkaline-lr06-aa-4up/,Т-000094958,H_no_group,H_no_group,H_no_group,energizer,Батарейка Energizer Alkaline LR06 AA 4уп,2024-08-29,12.0,NaN,0.0,NaN,бюджет,1299.916667,В наличии,8.0,3.0,1.0,0.0,NaN
1,https://hi-tech.md/televizory-i-elektronika/televizory/televizor-hisense-65e7nq-pro/,Т-000094928,H_no_group,H_no_group,H_no_group,hisense,телевизор Hisense 65E7NQ PRO,2024-08-29,15599.0,NaN,0.0,NaN,элитный,1299.916667,В наличии,8.0,3.0,1.0,0.0,NaN


In [117]:
dfh['error_group'] = 0
dfh.loc[dfh['_subgroup'].isna(), 'error_group'] = 1
dfh.loc[dfh['_subgroup']=='err0', 'error_group'] = 1
dfh.loc[dfh['_subgroup']=='err1', 'error_group'] = 1

Создание столбцов с флагом ошибки для товаров, для которых нет данных по остаткам хотябы в одном магазине

In [118]:
dfh.loc[dfh['error_group']==1 , ['_category',
       '_subcategory', '_group', '_subgroup']] = 'H_no_group'

удаление наличия товара в магазинах и наименовании группы если у товара есть ошибки в этих данны

Изменение типов данных столбцов наличия товара в определенном магазине на тип данных `float`  и создания столбцов по наличию товара в магазинах городов

In [119]:
dfh = dfh[['url', 'id', '_category', '_subcategory', '_group', '_subgroup', 'title',
               'date',   'price', 'old_price', 'availability', 'error_group', ]]

изменение порядка располажения столбцов

In [120]:
preprocessing(dfh)

таблица имеет следующий вид:


,url,id,_category,_subcategory,_group,_subgroup,title,date,price,old_price,availability,error_group
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energizer-alkaline-lr06-aa-4up/,Т-000094958,H_no_group,H_no_group,H_no_group,H_no_group,Батарейка Energizer Alkaline LR06 AA 4уп,2024-08-29,12.0,NaN,В наличии,1
1,https://hi-tech.md/televizory-i-elektronika/televizory/televizor-hisense-65e7nq-pro/,Т-000094928,H_no_group,H_no_group,H_no_group,H_no_group,телевизор Hisense 65E7NQ PRO,2024-08-29,15599.0,NaN,В наличии,1
2,https://hi-tech.md/bytovaya-tehnika/tehnika-dlya-krasoty/vesy-napolnye/vesy-polaris-pws-1885dg-180kg/,Т-000063843,H_no_group,H_no_group,H_no_group,H_no_group,весы Polaris PWS 1885DG 180кг,2024-08-29,340.0,NaN,Ожидается,1
3,https://hi-tech.md/bytovaya-tehnika/stiralnye-i-sushilnye-mashiny/stiralno-sushilnye-mashiny/stiralnaya-mashina-s-sushkoy-samsung-wd80t554cbt-ua/,Т-000095158,H_no_group,H_no_group,H_no_group,H_no_group,Стиральная машина с сушкой SAMSUNG WD80T554CBT/UA,2024-08-29,14599.0,NaN,Ожидается,1
4,https://hi-tech.md/tovary-dlya-doma/servirovka-stola/stolovye-servizy/luminarc-diwali-noir-bulonnica-0-75l-4555-ucenka/,Т-000095044,H_no_group,H_no_group,H_no_group,H_no_group,"Luminarc DIWALI Noir бульонница 0,75л (4555) уценка",2024-08-29,39.0,43.0,В наличии,1


****************************************************************************************************
таблица имеет 9266 сторок и 12 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:8
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                0
id                 0
_category          0
_subcategory       0
_group             0
_subgroup          0
title              0
date               0
price              0
old_price       6921
availability       0
error_group        0
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url              0.0
id               0.0
_category        0.0
_subcategory     0.0
_group           0.0
_subgroup        0.0
title            0.0
date             0.0
price            0.0
old_price       74.7
availability     0.0
error_group      0.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                     object
id                      object
_category               object
_subcategory            object
_group                  object
_subgroup               object
title                   object
date            datetime64[ns]
price                  float64
old_price              float64
availability            object
error_group              int64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9266 entries, 0 to 9265
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   url           9266 non-null   object        
 1   id            9266 non-null   object        
 2   _category     9266 non-null   object        
 3   _subcategory  9266 non-null   object        
 4   _group        9266 non-null   object        
 5   _subgroup     9266 non-null   object        
 6   title         9266 non-null   object        
 7   date          9266 non-null   datetime64[ns]
 8   price         9266 non-null   float64       
 9   old_price     2345 non-null   float64       
 1

None

просмотр данных после парсинга

# Группировка товара

- разобраться с креслами в ногруп

In [121]:
dfh.loc[dfh['_category']=='H_товар_без_акции', '_category'] = 'H_no_group'
dfh.loc[dfh['_category']=='H_товар_по_акции', '_category'] = 'H_no_group'
dfh.loc[dfh['_subcategory']=='H_мультимедиа', '_subcategory'] = 'H_аудиосистемы'
dfh.loc[dfh['_group']=='H_акустические_системы', '_group'] = 'H_акустика'
dfh.loc[dfh['_group']=='H_музыкальная_техника', '_group'] = 'H_акустика'
dfh.loc[dfh['_subcategory']=='H_умный_дом_с_яндекс','_subcategory'] = 'H_аудиосистемы'
dfh.loc[dfh['_group']=='H_яндекс_станции', '_group'] = 'H_акустика'
dfh.loc[dfh['_group']=='H_airpods', '_subcategory'] = 'H_аудиосистемы'
dfh.loc[dfh['_group']=='H_airpods', '_group'] = 'H_наушники'
dfh.loc[dfh['_group']=='H_apple_watch', '_group'] = 'H_часы_и_браслеты'
dfh.loc[dfh['_group']=='H_iphone', '_group'] = 'H_смартфоны'
dfh.loc[dfh['_group']=='H_ipad', '_group'] = 'H_планшеты'
dfh.loc[dfh['_group']=='H_macbook', '_group'] = 'H_ноутбуки'
dfh.loc[dfh['_group']=='H_imac', '_group'] = 'H_моноблоки'
dfh.loc[dfh['_group']=='H_часы_и_браслеты', '_subcategory'] = 'H_гаджеты'
dfh.loc[dfh['_group']=='H_смартфоны', '_subcategory'] = 'H_смартфоны_и_аксессуары'
dfh.loc[dfh['_group']=='H_планшеты', '_subcategory'] = 'H_готовые_решения'
dfh.loc[dfh['_group']=='H_ноутбуки', '_subcategory'] = 'H_готовые_решения'
dfh.loc[dfh['_group']=='H_моноблоки', '_subcategory'] = 'H_готовые_решения'
dfh.loc[dfh['_subcategory']=='H_стиральные_и_сушильные_машины', '_group'] = 'H_стиральные_машины'
dfh.loc[dfh['_subgroup']=='H_сушильные_машины', '_group'] = 'H_сушильные_машины'
dfh.loc[dfh['_subgroup']=='H_наклейки', '_group'] = 'H_наклейки'
dfh.loc[dfh['_subgroup']=='H_подставки', '_group'] = 'H_подставки'
dfh.loc[dfh['_subgroup']=='H_замки', '_group'] = 'H_замки'
dfh.loc[dfh['_subgroup']=='H_usb_hub', '_group'] = 'H_usb_hub'
dfh.loc[dfh['_subgroup']=='H_сумки_и_чехлы', '_group'] = 'H_сумки_и_чехлы'
dfh.loc[dfh['_subgroup']=='H_защитные_стекла', '_group'] = 'H_защитные_стекла'
dfh.loc[dfh['_subcategory']=='H_мониторы','_group'] = 'H_мониторы'

In [122]:
for i in dfh['_category'].unique():
    t = dfh[dfh['_category']==i]
    print(i)
    print(t['_subcategory'].unique())
    print('-'*100)

H_no_group
['H_no_group']
----------------------------------------------------------------------------------------------------
H_бытовая_техника
['H_климатическая_техника' 'H_крупная_техника_для_кухни'
 'H_мелкая_техника_для_кухни' 'H_приготовление_напитков'
 'H_пылесосы_и_аксессуары' 'H_стиральные_и_сушильные_машины'
 'H_техника_для_красоты' 'H_товары_для_ухода_за_одеждой']
----------------------------------------------------------------------------------------------------
H_компьютерная_техника
['H_аудиосистемы' 'H_аксессуары' 'H_видеотехника' 'H_готовые_решения'
 'H_комплектующие' 'H_крепления' 'H_мониторы' 'H_оргтехника'
 'H_расходные_материалы']
----------------------------------------------------------------------------------------------------
H_телевизоры_и_электроника
['H_аудиосистемы' 'H_батарейки_и_зарядные_устройства'
 'H_смартфоны_и_аксессуары' 'H_телевизоры']
----------------------------------------------------------------------------------------------------
H_товары_для_д

In [123]:
for i in dfh['_subcategory'].unique():
    t = dfh[dfh['_subcategory']==i]
    print(i)
    print(t['_group'].unique())
    print('-'*100)

H_no_group
['H_no_group']
----------------------------------------------------------------------------------------------------
H_климатическая_техника
['H_кондиционеры_кассетные']
----------------------------------------------------------------------------------------------------
H_крупная_техника_для_кухни
['H_варочные_поверхности' 'H_вытяжки' 'H_духовые_шкафы'
 'H_морозильники_и_лари' 'H_плиты_газовые_и_электрические'
 'H_посудомоечные_машины' 'H_холодильники' 'H_электропечи_и_духовки']
----------------------------------------------------------------------------------------------------
H_мелкая_техника_для_кухни
['H_весы_кухонные']
----------------------------------------------------------------------------------------------------
H_приготовление_напитков
['H_электрические_чайники' 'H_термосы_термокружки_термосумки']
----------------------------------------------------------------------------------------------------
H_пылесосы_и_аксессуары
['H_пылесосы']
-----------------------------

просмотр групп товаров в подкатегориях

In [124]:
for i in dfh['_group'].unique():
    t = dfh[dfh['_group']==i]
    print(i)
    print(t['_subgroup'].unique())
    print('-'*100)

H_no_group
['H_no_group']
----------------------------------------------------------------------------------------------------
H_кондиционеры_кассетные
['H_кондиционеры_кассетные']
----------------------------------------------------------------------------------------------------
H_варочные_поверхности
['H_варочные_поверхности']
----------------------------------------------------------------------------------------------------
H_вытяжки
['H_вытяжки']
----------------------------------------------------------------------------------------------------
H_духовые_шкафы
['H_духовые_шкафы']
----------------------------------------------------------------------------------------------------
H_морозильники_и_лари
['H_морозильные_лари']
----------------------------------------------------------------------------------------------------
H_плиты_газовые_и_электрические
['H_газовые_плиты' 'H_комбинированные_плиты']
---------------------------------------------------------------------------------

In [125]:
dfh._category.unique()

array(['H_no_group', 'H_бытовая_техника', 'H_компьютерная_техника',
       'H_телевизоры_и_электроника', 'H_товары_для_дома'], dtype=object)

In [126]:
dfh._subcategory.unique()

array(['H_no_group', 'H_климатическая_техника',
       'H_крупная_техника_для_кухни', 'H_мелкая_техника_для_кухни',
       'H_приготовление_напитков', 'H_пылесосы_и_аксессуары',
       'H_стиральные_и_сушильные_машины', 'H_техника_для_красоты',
       'H_товары_для_ухода_за_одеждой', 'H_аудиосистемы', 'H_аксессуары',
       'H_видеотехника', 'H_готовые_решения', 'H_комплектующие',
       'H_крепления', 'H_мониторы', 'H_оргтехника',
       'H_расходные_материалы', 'H_батарейки_и_зарядные_устройства',
       'H_смартфоны_и_аксессуары', 'H_телевизоры', 'H_бытовая_химия',
       'H_кухонные_принадлежности', 'H_приготовление_пищи',
       'H_сервировка_стола'], dtype=object)

In [127]:
dfh._group.unique()

array(['H_no_group', 'H_кондиционеры_кассетные', 'H_варочные_поверхности',
       'H_вытяжки', 'H_духовые_шкафы', 'H_морозильники_и_лари',
       'H_плиты_газовые_и_электрические', 'H_посудомоечные_машины',
       'H_холодильники', 'H_электропечи_и_духовки', 'H_весы_кухонные',
       'H_электрические_чайники', 'H_пылесосы', 'H_сушильные_машины',
       'H_стиральные_машины', 'H_весы_напольные',
       'H_выпрямители_и_плойки_для_волос', 'H_фены_для_волос',
       'H_отпариватели_для_одежды', 'H_парогенераторы', 'H_наушники',
       'H_сумки_и_чехлы', 'H_видеорегистраторы', 'H_планшеты',
       'H_корпуса', 'H_материнские_платы', 'H_модули_памяти',
       'H_крепления', 'H_мониторы', 'H_принтеры_и_мфу', 'H_зипы',
       'H_саундбары', 'H_батарейки_и_зарядные_устройства', 'H_смартфоны',
       'H_телевизоры', 'H_средства_для_стирки', 'H_ножи',
       'H_ёмкости_для_сыпучих_и_контейнеры_для_еды',
       'H_термосы_термокружки_термосумки', 'H_сковороды', 'H_сотейники',
       'H_прочие_пре

In [128]:
dfh._subgroup.unique()

array(['H_no_group', 'H_кондиционеры_кассетные', 'H_варочные_поверхности',
       'H_вытяжки', 'H_духовые_шкафы', 'H_морозильные_лари',
       'H_газовые_плиты', 'H_комбинированные_плиты',
       'H_посудомоечные_машины', 'H_side_by_side_и_french_door',
       'H_с_верхней_морозильной_камерой',
       'H_с_нижней_морозильной_камерой', 'H_широкие_холодильники',
       'H_электропечи_и_духовки', 'H_весы_кухонные',
       'H_электрические_чайники', 'H_пылесосы_вертикальные',
       'H_сушильные_машины', 'H_фронтальные_стиральные_машины',
       'H_весы_напольные', 'H_выпрямители_и_плойки_для_волос',
       'H_фены_для_волос', 'H_отпариватели_для_одежды',
       'H_парогенераторы', 'H_airpods', 'H_сумки_и_чехлы',
       'H_видеорегистраторы', 'H_планшеты', 'H_корпуса', 'H_intel',
       'H_материнские_платы', 'H_dimm', 'H_sodimm', 'H_крепления',
       'H_плоский', 'H_вкладыши', 'H_мфу', 'H_зипы', 'H_саундбары',
       'H_батарейки_и_зарядные_устройства', 'H_смартфоны', 'H_телевизоры',
   

# Подготовка анализа

In [129]:
dfh.head(2)

,url,id,_category,_subcategory,_group,_subgroup,title,date,price,old_price,availability,error_group
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energizer-alkaline-lr06-aa-4up/,Т-000094958,H_no_group,H_no_group,H_no_group,H_no_group,Батарейка Energizer Alkaline LR06 AA 4уп,2024-08-29,12.0,NaN,В наличии,1
1,https://hi-tech.md/televizory-i-elektronika/televizory/televizor-hisense-65e7nq-pro/,Т-000094928,H_no_group,H_no_group,H_no_group,H_no_group,телевизор Hisense 65E7NQ PRO,2024-08-29,15599.0,NaN,В наличии,1


def lower(x):
    return x.lower()
brend['names'] = brend['name'].apply(lower)
brend_up = brend.name.unique()
brend_low = brend.names.unique()

Создаем список брендов

def brends(x):
    xx = x.replace('\xa0', ' ')
    xx = xx.split(' ')    
    for i in range(len(xx)-1):
        if xx[i].lower() in brend_low:
            return xx[i].lower()
        elif (str(xx[i].lower())+' '+str(xx[i+1].lower())) in brend_low:
            return (str(xx[i].lower())+' '+str(xx[i+1].lower()))    
    else:    
        return 'другой'
tqdm.pandas()
df['brend'] = df['title'].progress_apply(brends)

In [130]:
tqdm.pandas()
dfh['brend'] = dfh['title'].progress_apply(brends)

  0%|          | 0/9266 [00:00<?, ?it/s]

создаем столбец с брендами

In [131]:
dfh['action_sale'] = 0
dfh.loc[dfh['old_price'] > 0,'action_sale'] = 1

Создаем столбец с флагом акции

In [132]:
dfh['sale'] = (100 - ((dfh['price']*100)/dfh['old_price'])).round(2) 

создаем столбец показывающий скидку в процентах

In [133]:
def h_count_group(x):
    return dfh[dfh['_group']==x]['id'].count()    
tqdm.pandas()
dfh['count_group'] = dfh['_group'].progress_apply(h_count_group)

  0%|          | 0/9266 [00:00<?, ?it/s]

создание столбца с количеством товара в группе

In [134]:
def h_count_group_presence(x):
    return dfh[(dfh['availability']=='В наличии')&(dfh['_group']==x)]['id'].count()   
tqdm.pandas()
dfh['activ_count_group'] = dfh['_group'].progress_apply(h_count_group_presence)

  0%|          | 0/9266 [00:00<?, ?it/s]

создание столбца с количеством товара, который есть в наличии, в группе

In [135]:
def h_dif_price(x):    
    max_price = dfh[(dfh['availability']=='В наличии')&(dfh['_group']==x)]['price'].max()
    min_price = dfh[(dfh['availability']=='В наличии')&(dfh['_group']==x)]['price'].min() 
    return max_price / min_price
tqdm.pandas()
dfh['dif_price'] = dfh['_group'].progress_apply(h_dif_price)

  0%|          | 0/9266 [00:00<?, ?it/s]

создам столбец с отношением самой дорогой цены в группе к самой дешевой.

In [136]:
dfh['group_prise'] = dfh.groupby('_group')['price'].transform(lambda x: pd.cut(x, bins = 4, labels=["бюджет", "стандарт", 'дорогой', "элитный"]))

создание столбца с ценовой категорией товара

In [137]:
dfh = dfh[['url', 'id', '_category', '_subcategory', '_group', '_subgroup', 'brend', 
         'title', 'date', 'price', 'old_price', 'action_sale', 'sale', 'group_prise', 'dif_price',
         'availability', 'count_group', 'activ_count_group','error_group']]

In [138]:
dfh.drop('_subgroup', axis= 1 , inplace= True )

In [139]:
dfh['tir'] = 0

# объединение

In [140]:
df = pd.concat([dfh, dft], ignore_index=True)

In [141]:
df = df.drop_duplicates().reset_index(drop=True)

In [142]:
df.to_csv('csv/hay_tir_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)  

In [143]:
preprocessing(df)

таблица имеет следующий вид:


,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energizer-alkaline-lr06-aa-4up/,Т-000094958,H_no_group,H_no_group,H_no_group,energizer,Батарейка Energizer Alkaline LR06 AA 4уп,2024-08-29,12.0,NaN,0,NaN,бюджет,59999.0,В наличии,9178,8415,1,0
1,https://hi-tech.md/televizory-i-elektronika/televizory/televizor-hisense-65e7nq-pro/,Т-000094928,H_no_group,H_no_group,H_no_group,hisense,телевизор Hisense 65E7NQ PRO,2024-08-29,15599.0,NaN,0,NaN,бюджет,59999.0,В наличии,9178,8415,1,0
2,https://hi-tech.md/bytovaya-tehnika/tehnika-dlya-krasoty/vesy-napolnye/vesy-polaris-pws-1885dg-180kg/,Т-000063843,H_no_group,H_no_group,H_no_group,polaris,весы Polaris PWS 1885DG 180кг,2024-08-29,340.0,NaN,0,NaN,бюджет,59999.0,Ожидается,9178,8415,1,0
3,https://hi-tech.md/bytovaya-tehnika/stiralnye-i-sushilnye-mashiny/stiralno-sushilnye-mashiny/stiralnaya-mashina-s-sushkoy-samsung-wd80t554cbt-ua/,Т-000095158,H_no_group,H_no_group,H_no_group,samsung,Стиральная машина с сушкой SAMSUNG WD80T554CBT/UA,2024-08-29,14599.0,NaN,0,NaN,бюджет,59999.0,Ожидается,9178,8415,1,0
4,https://hi-tech.md/tovary-dlya-doma/servirovka-stola/stolovye-servizy/luminarc-diwali-noir-bulonnica-0-75l-4555-ucenka/,Т-000095044,H_no_group,H_no_group,H_no_group,luminarc,"Luminarc DIWALI Noir бульонница 0,75л (4555) уценка",2024-08-29,39.0,43.0,1,9.3,бюджет,59999.0,В наличии,9178,8415,1,0


****************************************************************************************************
таблица имеет 21186 сторок и 19 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                      0
id                       0
_category                0
_subcategory             0
_group                   0
brend                    0
title                    0
date                     0
price                    0
old_price            17029
action_sale              0
sale                 17029
group_prise              0
dif_price               36
availability             0
count_group              0
activ_count_group        0
error_group              0
tir                      0
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url                   0.0
id                    0.0
_category             0.0
_subcategory          0.0
_group                0.0
brend                 0.0
title                 0.0
date                  0.0
price                 0.0
old_price            80.4
action_sale           0.0
sale                 80.4
group_prise           0.0
dif_price             0.2
availability          0.0
count_group           0.0
activ_count_group     0.0
error_group           0.0
tir                   0.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                          object
id                           object
_category                    object
_subcategory                 object
_group                       object
brend                        object
title                        object
date                 datetime64[ns]
price                       float64
old_price                   float64
action_sale                   int64
sale                        float64
group_prise                category
dif_price                   float64
availability                 object
count_group                   int64
activ_count_group             int64
error_group                   int64
tir                           int64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21186 entries, 0 to 21185
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   url                21186 non-null  object        
 1   id                 21186 non-null  object        
 2   _category          21186 non-null  object        
 3   _subcategory       21186 non-null  object        
 4   _group             21186 non-null  object        
 5   brend              21186 non-null  object        
 6   title              21186 non-null  object        
 7   date               21186 non-null  datetime64[ns]
 8   price              21186 non-null  float64   

None

In [144]:
t = df[df['tir']==1]

In [145]:
len(t)

11928

In [146]:
t._group.unique()

array(['T_автомагнитола', 'T_автопылесос', 'T_авторегистратор',
       'T_автохолодильник', 'T_динамики', 'T_компрессоры',
       'T_крепления_смартфонов/планшетов', 'T_модуляторы_fm',
       'T_навигаторы_gps', 'T_видеокамеры_аналоговые',
       'T_видеокамеры_цифровые', 'T_видеокамеры_эндоскопические',
       'T_видеоняни', 'T_видеорегистраторы_аналоговые',
       'T_видеорегистраторы_портативные', 'T_видеорегистраторы_цифровые',
       'T_комплекты_видеонаблюдения', 'T_видеодомофоны', 'T_видеозвонок',
       'T_вызывные_панели', 'T_мониторы', 'T_кабель_гофра',
       'T_коробки_монтажные', 'T_кронштейны', 'T_разъемы_и_переходники',
       'T_резервные_источники_питания', 'T_солнечные_панели',
       'T_термокожухи', 'T_адаптеры_poe', 'T_инжекторы_poe',
       'T_сплитер_poe', 'T_удлинители_poe', 'T_датчики', 'T_извещатели',
       'T_модули_расширения', 'T_оповещатели', 'T_панели',
       'T_пульты_кнопки', 'T_доводчики_дверные', 'T_замки',
       'T_карты_доступа', 'T_кнопки', 'T_к

In [147]:
t[t['_group']=='T_смартфоны']

,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir
16521,https://tiraet.com/catalog/2873/smartfone/smartfon_samsung_galaxy_a10_2_32gb_black/,80841,T_мобильные_телефоны,T_смартфоны,T_смартфоны,samsung,Смартфон Samsung Galaxy A10 BLACK 2/32GB (SM-A105F) (Без связи IDC),2024-08-29,1250.0,1499.0,1,16.61,бюджет,21.992,Достаточно,220,220,0,1
16522,https://tiraet.com/catalog/2873/smartfone/smartfon_realme_c31_silver_4_64gb_rmx3501_6_5_1600_720_ips_2_sim_karty_nfc_kamery_50_2_2_5_mpiks/,82081,T_мобильные_телефоны,T_смартфоны,T_смартфоны,realme,Смартфон Realme C31 Silver 4/64GB (RMX3501) (Без связи IDC),2024-08-29,1749.0,NaN,0,NaN,бюджет,21.992,Мало,220,220,0,1
16523,https://tiraet.com/catalog/2873/smartfone/smartfon_samsung_galaxy_a10s_2_32gb_black/,80843,T_мобильные_телефоны,T_смартфоны,T_смартфоны,samsung,Смартфон Samsung Galaxy A10S BLACK 2/32GB (SM-A107F/DS),2024-08-29,1635.0,2190.1,1,25.35,бюджет,21.992,Мало,220,220,0,1
16524,https://tiraet.com/catalog/2873/smartfone/smartfon_blackview_bv4800_3_64gb_orange_6_56_1612x720_ips_2_nano_sim_13_5mp/,85747,T_мобильные_телефоны,T_смартфоны,T_смартфоны,blackview,"Смартфон BLACKVIEW BV4800 3/64GB ORANGE 6.56' (1612x720) IPS, 2 nano SIM, 13/5Mp",2024-08-29,2249.9,NaN,0,NaN,бюджет,21.992,Много,220,220,0,1
16525,https://tiraet.com/catalog/2873/smartfone/smartfon_blackview_bv4800_3_64gb_green_6_56_1612x720_ips_2_nano_sim_13_5mp/,85746,T_мобильные_телефоны,T_смартфоны,T_смартфоны,blackview,"Смартфон BLACKVIEW BV4800 3/64GB GREEN 6.56' (1612x720) IPS, 2 nano SIM, 13/5Mp",2024-08-29,1999.0,2249.9,1,11.15,бюджет,21.992,Много,220,220,0,1
16526,https://tiraet.com/catalog/2873/smartfone/smartfon_blackview_bv4800_3_64gb_black_6_56_1612x720_ips_2_nano_sim_13_5mp/,85745,T_мобильные_телефоны,T_смартфоны,T_смартфоны,blackview,"Смартфон BLACKVIEW BV4800 3/64GB BLACK 6.56' (1612x720) IPS, 2 nano SIM, 13/5Mp",2024-08-29,1999.0,2249.9,1,11.15,бюджет,21.992,Много,220,220,0,1
16527,https://tiraet.com/catalog/2873/smartfone/smartfon_realme_c31_dark_green_3_32gb_rmx3501/,80536,T_мобильные_телефоны,T_смартфоны,T_смартфоны,realme,Смартфон Realme C31 Dark Green 3/32GB (RMX3501,2024-08-29,1990.1,2290.0,1,13.10,бюджет,21.992,Мало,220,220,0,1
16528,https://tiraet.com/catalog/2873/smartfone/smartfon_realme_c30_2gb_32gb_black/,80535,T_мобильные_телефоны,T_смартфоны,T_смартфоны,realme,Смартфон Realme C30 Black 2/32GB (RMX3581),2024-08-29,1850.0,2290.0,1,19.21,бюджет,21.992,Мало,220,220,0,1
16529,https://tiraet.com/catalog/2873/smartfone/smartfon_samsung_galaxy_a20_3_32gb_black/,80859,T_мобильные_телефоны,T_смартфоны,T_смартфоны,samsung,Смартфон Samsung Galaxy A20 BLACK 3/32GB (SM-A205F/DS),2024-08-29,1990.1,2390.0,1,16.73,бюджет,21.992,Мало,220,220,0,1
16530,https://tiraet.com/catalog/2873/smartfone/smartfon_realme_narzo_50i_prime_4_64gb_mint_green/,80543,T_мобильные_телефоны,T_смартфоны,T_смартфоны,realme,Смартфон Realme Narzo 50i Prime Mint Green 4/64GB (RMX3506),2024-08-29,2439.0,NaN,0,NaN,бюджет,21.992,Достаточно,220,220,0,1


In [148]:
t.sort_values('sale', ascending=False).head(100)

,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group,tir
13402,https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/naklejki/Naklejka-dlja-noutbukov-A4-GSA-15D-Aloha-Daydlja-154-14-13/,20387,T_компьютерная_техника,T_аксессуары,T_наклейки_на_клавиатуру,другой,"Наклейка на крышку ноутбука A4-GSA-15D 'Aloha Day'(для 15.4', 14', 13')",2024-08-29,13.2,32.8,1,59.76,бюджет,2.272727,Мало,7,7,0,1
13401,https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/naklejki/naklejka-dlja-noutbuka-a4-gsl-17b--lux-leopard---brown-dlja-17/,27888,T_компьютерная_техника,T_аксессуары,T_наклейки_на_клавиатуру,другой,Наклейка на крышку ноутбука A4-GSL-17B 'Lux Leopard - Brown' (для 17'),2024-08-29,13.2,32.8,1,59.76,бюджет,2.272727,Мало,7,7,0,1
16792,https://tiraet.com/catalog/2873/smartfon_acsess/9922/tpu_chekhol_getman_ease_logo_usilennye_ugly_dlya_samsung_galaxy_a50_a505f_a50s_a30s_bestsvetnyy_proz/,81128,T_мобильные_телефоны,T_чехлы,T_чехлы,getman,TPU чехол GETMAN Ease logo усиленные углы для Samsung Galaxy A50 (A505F)/A50s/A30s (Бесцветный (прозрачный),2024-08-29,35.0,85.0,1,58.82,бюджет,36.828571,Мало,130,130,0,1
13309,https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/kovriki/mouse_pad_a4_tech_a4_gmbw_20en_endless_note_/,20381,T_компьютерная_техника,T_аксессуары,T_коврики,другой,MOUSE PAD A4-TECH A4-GMBW-20EN (коврик д/опт. мыши 'Endless Note'),2024-08-29,34.0,74.3,1,54.24,бюджет,39.933333,Мало,73,73,0,1
10908,https://tiraet.com/catalog/bytovaya/krupnaja-bytovaja-tehnika/pylesosy/pylesos-robot-polaris-pvcr-1012u-suhaja-uborkaamortkolesamoschn18vt/,55117,T_бытовая_техника,T_крупная_бытовая_техника,T_пылесосы,polaris,"Пылесос-робот Polaris PVCR 1012U (сухая уборка,аморт.колеса,мощн.18Вт)",2024-08-29,839.0,1699.0,1,50.62,бюджет,25.405769,Мало,95,95,0,1
14931,https://tiraet.com/catalog/Komplektujuschie/manipuljatory/klaviatury11/keyboard-sven-kb-g9700-igrovajamehanperekljuchrgb-podsvetusbblack/,56073,T_компьютерная_техника,T_манипуляторы,T_клавиатуры,sven,"KEYBOARD SVEN KB-G9700 игровая,механ.переключ,RGB подсвет,USB,black",2024-08-29,495.0,999.1,1,50.46,бюджет,34.471264,Мало,116,116,0,1
13476,https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/sumki-i-chehlynout/sumka-chehol-dlja-tablet-8-yac120-/,33362,T_компьютерная_техника,T_аксессуары,T_сумки_и_чехлы,другой,"Сумка-чехол для TABLET 8' YAC120 (229x171x25мм,Black/Grey,Neoprene)",2024-08-29,39.9,80.4,1,50.37,бюджет,25.918919,Достаточно,115,115,0,1
13487,https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/sumki-i-chehlynout/sumka-chehol-dlja-tablet-7-169-yac101/,34277,T_компьютерная_техника,T_аксессуары,T_сумки_и_чехлы,другой,"Сумка-чехол для TABLET 7' (16:9) YAC101(213x145x19мм,Black/Whit,Neopr)",2024-08-29,44.1,88.6,1,50.23,бюджет,25.918919,Мало,115,115,0,1
13484,https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/sumki-i-chehlynout/sumka-chehol-dlja-tablet-10-yac150-/,33364,T_компьютерная_техника,T_аксессуары,T_сумки_и_чехлы,другой,"Сумка-чехол для TABLET 10' YAC150 (288x195x30мм,Black/Grey,Neoprene)",2024-08-29,40.9,82.0,1,50.12,бюджет,25.918919,Много,115,115,0,1
13485,https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/sumki-i-chehlynout/sumka-chehol-dlja-tablet-10-yac140-/,33363,T_компьютерная_техника,T_аксессуары,T_сумки_и_чехлы,другой,"Сумка-чехол для TABLET 10' YAC140 (277x189x19мм,Black/Grey,Neoprene)",2024-08-29,40.9,82.0,1,50.12,бюджет,25.918919,Много,115,115,0,1
